# Strategy Actor-Critic

> CRLD actor-critic agents in strategy space  

In [ ]:
#| default_exp Agents/StrategyActorCritic

In [ ]:
#| hide
# Imports for the nbdev development environment
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import jax
from jax import jit
import jax.numpy as jnp

import numpy as np
import itertools as it
from functools import partial

from fastcore.utils import *

from pyCRLD.Agents.StrategyBase import strategybase
from pyCRLD.Utils.Helpers import *

In [ ]:
#| export
class stratAC(strategybase):
    """
    Class for CRLD-actor-critic agents in strategy space.
    """
    
    @partial(jit, static_argnums=(0,2))
    def RPEisa(self,
               Xisa,  # Joint strategy
               norm=False # normalize error around actions? 
               ) -> np.ndarray:  # RP/TD error
        """
        Compute reward-prediction/temporal-difference error for 
        strategy actor-critic dynamics, given joint strategy `Xisa`.
        """
        R = self.Risa(Xisa)
        Vis = self.Vis(Xisa, Risa=R)
        NextV = self.NextVisa(Xisa, Vis=Vis)

        n = jnp.newaxis
        E = self.pre[:,n,n]*R + self.gamma[:,n,n]*NextV - Vis[:,:,n]
        E *= self.beta[:,n,n]

        E = E - E.mean(axis=2, keepdims=True) if norm else E
        return E

    
    @partial(jit, static_argnums=0)
    def NextVisa(self,
                 Xisa,      # Joint strategy
                 Vis=None,  # Optional values for speed-up
                 Tss=None,  # Optional transition for speed-up
                 Ris=None,  # Optional reward for speed-up
                 Risa=None  # Optional reward for speed-up
                ) -> jnp.ndarray: # Next values
        """
        Compute strategy-average next value for agent `i`, current state `s` and action `a`.
        """
        Vis = self.Vis(Xisa, Ris=Ris, Tss=Tss, Risa=Risa) if Vis is None else Vis
        
        i = 0; a = 1; s = 2; s_ = 3
        j2k = list(range(6, 6+self.N-1))  # other agents
        b2d = list(range(6+self.N-1, 6+self.N-1 + self.N))  # all actions
        e2f = list(range(5+2*self.N, 5+2*self.N + self.N-1))  # all other acts

        sumsis = [[j2k[l], s, e2f[l]] for l in range(self.N-1)]  # sum inds
        otherX = list(it.chain(*zip((self.N-1)*[Xisa], sumsis)))
                                
        args = [self.Omega, [i]+j2k+[a]+b2d+e2f] + otherX +\
            [self.T, [s]+b2d+[s_], Vis, [i, s_], [i, s, a]]
        return jnp.einsum(*args, optimize=self.opti)

Note, `choice_intensities` are not required for actor-critic learning and have no other effect than scaling the `learning_rates`. Hence the default value of `1`.

In [ ]:
show_doc(stratAC.RPEisa)

In [ ]:
show_doc(stratAC.NextVisa)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()